In [209]:
from mtm026 import *

P0 = 1          #[N]
L1 = 5          #[m]
L2 = 4.5        #[m]
a = 0.05        #[m]
t = 0.002       #[m]
E = 200e9       #[Pa]
v = 0.3

I_1 = 4.7366e-7
I_2 = 1.4771e-6

In [ ]:
K = np.zeros([16, 16])
K_sig = np.zeros([16, 16])

ke1 = Ke_balk(E*I_2, L2)
ke2 = Ke_balk(E*I_2, L2)
ke3 = Ke_balk(E*I_2, L2)
ke4 = Ke_balk(E*I_2, L2)
ke5 = Ke_balk(E*I_1, L1)
ke6 = Ke_balk(E*I_1, L1)

ke1_sig = Ke_sigma_balk(P0, L2)
ke2_sig = Ke_sigma_balk(P0, L2)
ke4_sig = Ke_sigma_balk(P0, L2)
ke3_sig = Ke_sigma_balk(P0, L2)
ke5_sig = Ke_sigma_balk(P0, L1)
ke6_sig = Ke_sigma_balk(P0, L1)

assem(K, ke1, dofs=[1,2,3,4])
assem(K, ke2, dofs=[3,4,5,6])
assem(K, ke3, dofs=[7,8,9,10])
assem(K, ke4, dofs=[9,10,11,12])
assem(K, ke5, dofs=[3,4,9,10])
assem(K, ke6, dofs=[5,6,11,12])

# assem(K_sig, ke1_sig, dofs=[1,2,3,4])
# assem(K_sig, ke2_sig, dofs=[3,4,5,6])
# assem(K_sig, ke3_sig, dofs=[7,8,9,10])
# assem(K_sig, ke4_sig, dofs=[9,10,11,12])
# assem(K_sig, ke5_sig, dofs=[3,4,9,10])
# assem(K_sig, ke6_sig, dofs=[5,6,11,12])

displayvar("K", K)
displayvar("K_\sigma", K_sig)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [211]:
# Definiera våra fria frihetsgrader
free_dofs = [3,4,5,6,7,8,10,12]

# Reducera till enbart fria frihetsgrader
K_red = extract_block(K, free_dofs, free_dofs)
K_sigma_red = extract_block(K_sig, free_dofs, free_dofs)

In [212]:
# Lös egenvärdesproblemet (K - α K_σ)n = 0
# Behöver inte skriva om till ett vanligt egenvärdesproblem för det finns lösare för det 
# generaliserade egenvärdesproblemet 

alpha, knäckningsmoder = eig(K_red, K_sigma_red) # => egenvektorer & egenvärden 
# vill man enbart ha realdelen till α kan man skriva α.real 

# Skriv ut lastmultiplikatorerna 
print(f"α = {alpha}")
print(f"Kristisk last P_kr = {alpha[0]*P0} EI/L^2")

α = [660767.70873095+0.j 472275.08595468+0.j 314577.76192549+0.j
 226479.08381021+0.j 147745.76799703+0.j  70590.26236611+0.j
  22155.51962757+0.j  42226.95129288+0.j]
Kristisk last P_kr = (660767.708730946+0j) EI/L^2


In [214]:
import numpy as np
from mtm026 import *
from scipy.linalg import eig

# Parametrar
P0 = 1          # [N]
L1 = 5.0        # [m] - horisontella
L2 = 4.5        # [m] - vertikala
a = 0.05        # [m]
t = 0.002       # [m]
E = 200e9       # [Pa]

# Tröghetsmoment
I_1 = 4.7366e-7  # horisontella
I_2 = 1.4771e-6  # vertikala

# Initiera globala matriser (16x16 p.g.a. 8 noder × 2 DOFs)
K = np.zeros((16, 16))
K_sigma = np.zeros((16, 16))

# Skapa elementstyvhetsmatriser
ke = [
    Ke_balk(E*I_2, L2),  # e1
    Ke_balk(E*I_2, L2),  # e2
    Ke_balk(E*I_2, L2),  # e3
    Ke_balk(E*I_2, L2),  # e4
    Ke_balk(E*I_1, L1),  # e5
    Ke_balk(E*I_1, L1),  # e6
]

ke_sigma = [
    Ke_sigma_balk(P0, L2),  # e1
    Ke_sigma_balk(P0, L2),  # e2
    Ke_sigma_balk(P0, L2),  # e3
    Ke_sigma_balk(P0, L2),  # e4
    Ke_sigma_balk(0, L1),   # e5
    Ke_sigma_balk(0, L1),   # e6
]

# Assemblering enligt figur 1.2 i uppgiften (med rätt dofs-index)
assem(K, ke[0], dofs=[1,2,3,4])    # e1
assem(K, ke[1], dofs=[3,4,5,6])    # e2
assem(K, ke[2], dofs=[7,8,9,10])   # e3
assem(K, ke[3], dofs=[9,10,11,12]) # e4
assem(K, ke[4], dofs=[3,4,9,10])   # e5
assem(K, ke[5], dofs=[5,6,11,12])  # e6

assem(K_sigma, ke_sigma[0], dofs=[1,2,3,4])    # e1
assem(K_sigma, ke_sigma[1], dofs=[3,4,5,6])    # e2
assem(K_sigma, ke_sigma[2], dofs=[7,8,9,10])   # e3
assem(K_sigma, ke_sigma[3], dofs=[9,10,11,12]) # e4
assem(K_sigma, ke_sigma[4], dofs=[3,4,9,10])   # e5
assem(K_sigma, ke_sigma[5], dofs=[5,6,11,12])  # e6

# Reducera till frihetsgrader (endast v och φ i noder 1,2,4,5)
free_dofs = [1,2,3,4,7,8,9,10]  # 0-indexerade (Python)

K_red = K[np.ix_(free_dofs, free_dofs)]
K_sigma_red = K_sigma[np.ix_(free_dofs, free_dofs)]

# Lös egenvärdesproblem
alpha, vecs = eig(K_red, K_sigma_red)
alpha_real = np.real(alpha)
alpha_pos = alpha_real[alpha_real > 0]
alpha_kr = np.min(alpha_pos)

# Kritisk last
P_kr = alpha_kr * P0

# Skriv ut resultat
print(f"Lägsta kritiska lastmultiplikator α = {alpha_kr:.4f}")
print(f"Kritisk last P_kr = {P_kr:.2f} N")


Lägsta kritiska lastmultiplikator α = 14260.8613
Kritisk last P_kr = 14260.86 N
